In [1]:
import os
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
os.environ["CUDA_VISIBLE_DEVICES"]="5" #,1,2,3
from exit.utils import animate3d, t2m_bone
import numpy as np
import torch
from tqdm import tqdm
from exit.utils import visualize_2motions
from utils.motion_process import recover_from_ric
from generate import MMM
from dataset import dataset_TM_eval
from utils.word_vectorizer import WordVectorizer

In [2]:
w_vectorizer = WordVectorizer('./glove', 'our_vab')
val_loader = dataset_TM_eval.DATALoader('t2m', True, 32, w_vectorizer)

100%|██████████| 44/44 [00:00<00:00, 75.83it/s]

Pointer Pointing at 0


In [3]:
class Temp:
    def __init__(self, extra_args=None):
        print('mock:: opt')
        if extra_args is not None:
            for i in extra_args:
                self.__dict__[i] = extra_args[i]
args = Temp()

args.dataname = args.dataset_name = 't2m'
args.nb_code = 8192
args.code_dim = 32
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.quantizer = 'ema_reset'
args.mu = 0.99
args.clip_dim = 512


args.embed_dim_gpt = 1024
args.block_size = 51
args.num_layers = 9
args.n_head_gpt = 16
args.drop_out_rate = 0.1
args.ff_rate = 4

# Make you you already download pretrained model in read me section 2.3
args.resume_pth = './output/vq/2024-06-03-20-22-07_retrain/net_last.pth'
args.resume_trans = './output/t2m/2024-06-04-09-29-20_trans_name_b128/net_last.pth'
args.num_local_layer = 2

mock:: opt


In [4]:
import os
import json

import numpy as np
import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import models.vqvae as vqvae
import utils.losses as losses 
import options.option_vq as option_vq
import utils.utils_model as utils_model
from dataset import dataset_VQ, dataset_TM_eval
import utils.eval_trans as eval_trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from utils.word_vectorizer import WordVectorizer
from tqdm import tqdm
from exit.utils import animate3d, get_model, generate_src_mask, init_save_folder, visualize_2motions,plot_t2m,t2m_bone
from models.vqvae_sep import VQVAE_SEP
from models.vqvae_multi import VQVAE_MULTI
from models.vqvae_multi_v2 import VQVAE_MULTI_V2

In [6]:
from dataset import dataset_VQ
from models.vqvae_multi import VQVAE_MULTI
import torch
args.dataname='t2m'
resume_pthv2 = '/home/haoyum3/MMM/output/vq/2024-08-12-23-14-09_vq5_multi/net_last.pth'#/home/haoyum3/MMM/output/vq/2024-08-11-00-15-40_vq_multi/net_last.pth
resume_pth='/home/haoyum3/MMM/output/vq/2024-08-10-21-38-12_vq_multi/net_last.pth'
args.window_size = 64
args.vq_act = 'relu'
args.vq_norm = None
train_loader = dataset_VQ.DATALoader(args.dataname,
                                        1,
                                        window_size=args.window_size,
                                        unit_length=2**args.down_t)

train_loader_iter = dataset_VQ.cycle(train_loader)
netv2= VQVAE_MULTI_V2(args, ## use args to define different parameters in different quantizers
                    args.nb_code,#8192
                    40,#32
                    args.output_emb_width,#512
                    args.down_t,#2
                    args.stride_t,#2
                    args.width,#512
                    args.depth,#3
                    args.dilation_growth_rate,#3
                    args.vq_act,#'relu'
                    args.vq_norm,#None
                    {'mean': torch.from_numpy(train_loader.dataset.mean).cuda().float(), 
                    'std': torch.from_numpy(train_loader.dataset.std).cuda().float()},
                    True)
# net= VQVAE_MULTI(args, ## use args to define different parameters in different quantizers
#                     args.nb_code,#8192
#                     args.code_dim,#32
#                     args.output_emb_width,#512
#                     args.down_t,#2
#                     args.stride_t,#2
#                     args.width,#512
#                     args.depth,#3
#                     args.dilation_growth_rate,#3
#                     args.vq_act,#'relu'
#                     args.vq_norm,#None
#                     {'mean': torch.from_numpy(train_loader.dataset.mean).cuda().float(), 
#                     'std': torch.from_numpy(train_loader.dataset.std).cuda().float()},
#                     True)
# ckpt = torch.load(resume_pth, map_location='cpu')
# net.load_state_dict(ckpt['net'], strict=True)
# net.eval()
# net.cuda()

ckptv2= torch.load(resume_pthv2, map_location='cpu')
netv2.load_state_dict(ckptv2['net'], strict=True)
netv2.eval()
netv2.cuda()

100%|██████████| 23384/23384 [00:27<00:00, 839.72it/s] 


Total number of motions 20941


VQVAE_MULTI_V2(
  (decoder_left_arm): Decoder(
    (model): Sequential(
      (0): Conv1d(8, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU()
      (2): Sequential(
        (0): Resnet1D(
          (model): Sequential(
            (0): ResConv1DBlock(
              (norm1): Identity()
              (norm2): Identity()
              (activation1): ReLU()
              (activation2): ReLU()
              (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(9,), dilation=(9,))
              (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
            )
            (1): ResConv1DBlock(
              (norm1): Identity()
              (norm2): Identity()
              (activation1): ReLU()
              (activation2): ReLU()
              (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
              (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
            )
            (2): ResConv1DBlock(
             

In [10]:
std = np.load('./exit/t2m-std.npy')
mean = np.load('./exit/t2m-mean.npy')
args.sep_uplow = True
for nb_iter in tqdm(range(1, 2)):
    #[256,64,263]
    gt_motion = next(train_loader_iter)
    gt_motion = gt_motion.cuda().float() # bs, nb_joints, joints_dim, seq_len
    
    if args.sep_uplow:
        pred_motion, loss_commit, perplexity = netv2(gt_motion, idx_noise=0)
    elif args.sep_multi:
        pred_motion, loss_commit, perplexity = netv2(gt_motion, idx_noise=0)
    else:
        pred_motion, loss_commit, perplexity = netv2(gt_motion)
    #对于每个batch的motion进行可视化
    file_name = '_'+f'motion{nb_iter}'
    print(torch.equal(gt_motion, pred_motion))
    print('gt_motion:{}, pred_motion:{}'.format(gt_motion.shape, pred_motion.shape))
    print('gt_motion:')
    gt_motion = gt_motion[0, :64].detach().cpu().numpy() * val_loader.dataset.std + val_loader.dataset.mean
    gt_motion = recover_from_ric(torch.from_numpy(gt_motion).float(), 22).numpy()
    animate3d(gt_motion, BONE_LINK=t2m_bone, first_total_standard=63)
    print('pred_motion:')
    pred_motion = pred_motion[0, :64].detach().cpu().numpy() * val_loader.dataset.std + val_loader.dataset.mean
    pred_motion = recover_from_ric(torch.from_numpy(pred_motion).float(), 22).numpy()
    animate3d(pred_motion, BONE_LINK=t2m_bone, first_total_standard=63)

  0%|          | 0/1 [00:00<?, ?it/s]

False
gt_motion:torch.Size([1, 64, 263]), pred_motion:torch.Size([1, 64, 263])
gt_motion:


pred_motion:


100%|██████████| 1/1 [00:00<00:00,  1.68it/s]


: 

In [11]:
std = np.load('./exit/t2m-std.npy')
mean = np.load('./exit/t2m-mean.npy')
args.sep_uplow = True
for nb_iter in tqdm(range(1, 2)):
    #[256,64,263]
    gt_motion = next(train_loader_iter)
    gt_motion = gt_motion.cuda().float() # bs, nb_joints, joints_dim, seq_len
    
    if args.sep_uplow:
        pred_motion, loss_commit, perplexity = net(gt_motion, idx_noise=0)
    elif args.sep_multi:
        pred_motion, loss_commit, perplexity = net(gt_motion, idx_noise=0)
    else:
        pred_motion, loss_commit, perplexity = net(gt_motion)
    #对于每个batch的motion进行可视化
    file_name = '_'+f'motion{nb_iter}'
    print(torch.equal(gt_motion, pred_motion))
    print('gt_motion:{}, pred_motion:{}'.format(gt_motion.shape, pred_motion.shape))
    print('gt_motion:')
    gt_motion = gt_motion[0, :64].detach().cpu().numpy() * val_loader.dataset.std + val_loader.dataset.mean
    gt_motion = recover_from_ric(torch.from_numpy(gt_motion).float(), 22).numpy()
    animate3d(gt_motion, BONE_LINK=t2m_bone, first_total_standard=63)
    print('pred_motion:')
    pred_motion = pred_motion[0, :64].detach().cpu().numpy() * val_loader.dataset.std + val_loader.dataset.mean
    pred_motion = recover_from_ric(torch.from_numpy(pred_motion).float(), 22).numpy()
    animate3d(pred_motion, BONE_LINK=t2m_bone, first_total_standard=63)

  0%|          | 0/1 [00:00<?, ?it/s]

False
gt_motion:torch.Size([1, 64, 263]), pred_motion:torch.Size([1, 64, 263])
gt_motion:


pred_motion:


100%|██████████| 1/1 [00:00<00:00,  1.34it/s]
